In [1]:
import itertools
from tree_from_assoc_rules import list_all_factors
from controller import generate_decision_tree
from tabulate import tabulate
import datetime

In [2]:
# Define all factors - Ref(3)
all_factors = ["stay", "length_of_stay", "country", "review_group", "room_type"]

# Read file of possible values
possible_factor_values = list_all_factors()

In [3]:
# All possible permutations of all factors - Ref(4)
all_factors_permutations = []
for length in range(2, len(all_factors) + 1):
    all_factors_permutations += list(itertools.permutations(all_factors, length))

print(all_factors_permutations)

[('stay', 'length_of_stay'), ('stay', 'country'), ('stay', 'review_group'), ('stay', 'room_type'), ('length_of_stay', 'stay'), ('length_of_stay', 'country'), ('length_of_stay', 'review_group'), ('length_of_stay', 'room_type'), ('country', 'stay'), ('country', 'length_of_stay'), ('country', 'review_group'), ('country', 'room_type'), ('review_group', 'stay'), ('review_group', 'length_of_stay'), ('review_group', 'country'), ('review_group', 'room_type'), ('room_type', 'stay'), ('room_type', 'length_of_stay'), ('room_type', 'country'), ('room_type', 'review_group'), ('stay', 'length_of_stay', 'country'), ('stay', 'length_of_stay', 'review_group'), ('stay', 'length_of_stay', 'room_type'), ('stay', 'country', 'length_of_stay'), ('stay', 'country', 'review_group'), ('stay', 'country', 'room_type'), ('stay', 'review_group', 'length_of_stay'), ('stay', 'review_group', 'country'), ('stay', 'review_group', 'room_type'), ('stay', 'room_type', 'length_of_stay'), ('stay', 'room_type', 'country'), ('

In [4]:
# Hardcoded some constants
mcda_type = "rating"
hotel_name = "Amari Phuket"
rule_year = None
rule_month = None

In [5]:
print("all cases = " + str(len(all_factors_permutations)))
def collect_paths(node, route_nodes=[]):
    result = []
    has_child = len(node.children) > 0
    if has_child:
        for child_node in node.children:
            new_route_nodes = []
            new_route_nodes += route_nodes
            new_route_nodes.append(node)
            result += collect_paths(child_node, route_nodes=new_route_nodes)
    else:
        new_route_nodes = []
        new_route_nodes += route_nodes
        new_route_nodes.append(node)
        return [new_route_nodes]
    return result

# Generate decision tree for all permutations
count = 0
gen_count = 0
nodes = []
result_score_table = [["Outcome", "Factor_1", "Factor_2", "Factor_3", "Factor_4", "Factor_5", "Score"]]
for idx, permutation in enumerate(all_factors_permutations):
    # Calculate weights array
    factors_length = len(permutation)
    factor_weight = 1 / factors_length
    weights = [factor_weight for x in range(factors_length)]

    # Generate Decision Tree
    # print(permutation)
    gen_count += 1
    a = datetime.datetime.now()
    root_node = generate_decision_tree(mcda_type, list(permutation), weights, hotel_name=hotel_name,
                                       rule_year=rule_year, rule_month=rule_month)
    b = datetime.datetime.now()
    c = b - a
    print(str(gen_count) + ": " + str(c) + " - " + str(permutation))
    nodes.append(root_node)

    # Prototype: calculate score on each tree
    # print(RenderTree(root_node, style=AsciiStyle()).by_attr())
    paths = collect_paths(root_node)
    for p_idx, path in enumerate(paths):
        count += 1
        result_name = list(map(lambda k : k.name, path[1:]))
        weight_sum = sum(n.weight for n in path[1:])
        score_sum = sum(n.score for n in path[1:]) if weight_sum == 1 else 0
        result_row = [None] * 7
        result_row[0] = count
        result_row[1:5] = result_name + [None] * (5 - len(result_name))
        result_row[6] = score_sum
        result_score_table.append(result_row)

all cases = 320
1: 0:00:58.039750 - ('stay', 'length_of_stay')
2: 0:00:21.099972 - ('stay', 'country')
3: 0:00:19.018473 - ('stay', 'review_group')
4: 0:00:16.675163 - ('stay', 'room_type')
5: 0:00:05.155911 - ('length_of_stay', 'stay')
6: 0:00:02.673826 - ('length_of_stay', 'country')
7: 0:00:04.362000 - ('length_of_stay', 'review_group')
8: 0:00:02.876554 - ('length_of_stay', 'room_type')
9: 0:00:47.206017 - ('country', 'stay')
10: 0:00:44.418791 - ('country', 'length_of_stay')
11: 0:00:45.583035 - ('country', 'review_group')
12: 0:00:44.986655 - ('country', 'room_type')
13: 0:00:04.813688 - ('review_group', 'stay')
14: 0:00:03.672263 - ('review_group', 'length_of_stay')
15: 0:00:05.352265 - ('review_group', 'country')
16: 0:00:07.385300 - ('review_group', 'room_type')
17: 0:02:31.247381 - ('room_type', 'stay')
18: 0:02:32.487410 - ('room_type', 'length_of_stay')
19: 0:02:30.882460 - ('room_type', 'country')
20: 0:02:25.920745 - ('room_type', 'review_group')
21: 0:00:06.701705 - ('st

ConnectionError: HTTPConnectionPool(host='34.124.232.97', port=9000): Max retries exceeded with url: /rules/dss?antecedents=length_of_stay%3A1-7&consequents=country&hotel_name=Amari+Phuket (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000021549C0A880>: Failed to establish a new connection: [WinError 10065] A socket operation was attempted to an unreachable host'))

In [7]:
import csv

# print(tabulate(result_score_table, headers='firstrow', tablefmt='fancy_grid'))
with open("out.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(result_score_table)